In [1]:
import argparse
import os
import numpy as np
import math
import itertools
import datetime
import time

import sys

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

from models import *
from datasets import *

import torch.nn as nn
import torch.nn.functional as F
import torch

D_VAE_loss=[]
LR_loss=[]
G_loss=[]

In [ ]:

parser = argparse.ArgumentParser()
parser.add_argument("--epoch", type=int, default=0, help="トレーニングを開始するためのエポック")
parser.add_argument("--n_epochs", type=int, default=90, help="トレーニングエポック数")
parser.add_argument("--dataset_name", type=str, default="sample", help="データセットの名前")
parser.add_argument("--batch_size", type=int, default=16, help="バッチサイズ")
parser.add_argument("--lr", type=float, default=0.0002, help="adam: 学習速度")
parser.add_argument("--b1", type=float, default=0.5, help="adam: 勾配の減衰")
parser.add_argument("--b2", type=float, default=0.999, help="adam: decay of first order momentum of gradient")
parser.add_argument("--n_cpu", type=int, default=3, help="CPUスレッド数")
parser.add_argument("--img_height", type=int, default=256, help="画像サイズの高さ")
parser.add_argument("--img_width", type=int, default=256, help="画像サイズの横")
parser.add_argument("--channels", type=int, default=3, help="画像チャネル数")
parser.add_argument("--latent_dim", type=int, default=8, help="潜伏次元数（ノイズベクトル？）")
parser.add_argument("--sample_interval", type=int, default=500, help="interval between saving generator samples")
parser.add_argument("--checkpoint_interval", type=int, default=40, help="interval between model checkpoints")
parser.add_argument("--lambda_pixel", type=float, default=10, help="pixelwise loss weight")
parser.add_argument("--lambda_latent", type=float, default=0.5, help="latent loss weight")
parser.add_argument("--lambda_kl", type=float, default=0.01, help="kullback-leibler loss weight")
opt = parser.parse_args(args=[])
print(opt)

os.makedirs("images/%s" % opt.dataset_name, exist_ok=True)
os.makedirs("saved_models/%s" % opt.dataset_name, exist_ok=True)

cuda = True if torch.cuda.is_available() else False

input_shape = (opt.channels, opt.img_height, opt.img_width)



# Loss関数
mae_loss = torch.nn.L1Loss()

# Initialize generator, encoder and discriminators
generator = Generator(opt.latent_dim, input_shape)
encoder = Encoder(opt.latent_dim, input_shape)
D_VAE = MultiDiscriminator(input_shape)
D_LR = MultiDiscriminator(input_shape)

if cuda:
    print("CUDAが使えます")
    generator = generator.cuda()
    encoder.cuda()
    D_VAE = D_VAE.cuda()
    D_LR = D_LR.cuda()
    mae_loss.cuda()

if opt.epoch != 0:
    # Load pretrained models
    generator.load_state_dict(torch.load("saved_models/%s/generator_%d.pth" % (opt.dataset_name, opt.epoch)))
    encoder.load_state_dict(torch.load("saved_models/%s/encoder_%d.pth" % (opt.dataset_name, opt.epoch)))
    D_VAE.load_state_dict(torch.load("saved_models/%s/D_VAE_%d.pth" % (opt.dataset_name, opt.epoch)))
    D_LR.load_state_dict(torch.load("saved_models/%s/D_LR_%d.pth" % (opt.dataset_name, opt.epoch)))
else:
    # Initialize weights
    generator.apply(weights_init_normal)
    D_VAE.apply(weights_init_normal)
    D_LR.apply(weights_init_normal)

# Optimizers
optimizer_E = torch.optim.Adam(encoder.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))
optimizer_G = torch.optim.Adam(generator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))
optimizer_D_VAE = torch.optim.Adam(D_VAE.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))
optimizer_D_LR = torch.optim.Adam(D_LR.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))

################################################################

#画像の読み込み

###################################################################

#画像の前処理方法の設定
transforms_ = [
    transforms.Resize((opt.img_height, opt.img_width), Image.BICUBIC),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)),
]

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

# #trainに使う画像と、valに使う画像を分ける。
# train_A, train_B = make_datapath_list(opt)
# val_A, val_B = make_datapath_list(opt, phase="val")

# dataloader = DataLoader(
#     #ImageDatasetはDatasetクラスをオリジナルに改変したもの->datasets.py
#     ImageDataset(train_A, train_B, opt, transforms_=transforms_),
#     batch_size=opt.batch_size,
#     shuffle=True,
#     num_workers=opt.n_cpu,
# )

# val_dataloader = DataLoader(
#     ImageDataset(val_A, val_B, opt, transforms_=transforms_),
#     batch_size=opt.batch_size,
#     shuffle=False,
#     num_workers=1,
# )


dataloader = DataLoader(
    ImageDataset("./data/%s" % opt.dataset_name, input_shape, mode="train" ),
    batch_size=opt.batch_size,
    shuffle=True,
    num_workers=opt.n_cpu,
)
val_dataloader = DataLoader(
    ImageDataset("./data/%s" % opt.dataset_name, input_shape, mode="val"),
    batch_size=opt.batch_size,
    shuffle=False,
    num_workers=opt.n_cpu,
)

#####################################################

#画像保存関数

###################################################

def sample_images(batches_done):
    """Saves a generated sample from the validation set"""
    generator.eval()
    imgs = next(iter(val_dataloader))
    img_samples = None
    for img_A, img_B in zip(imgs["A"], imgs["B"]):
        # Repeat input image by number of desired columns
        real_A = img_A.view(1, *img_A.shape).repeat(opt.latent_dim, 1, 1, 1)
        real_A = Variable(real_A.type(Tensor))
        # Sample latent representations
        sampled_z = Variable(Tensor(np.random.normal(0, 1, (opt.latent_dim, opt.latent_dim))))
        # Generate samples
        fake_B = generator(real_A, sampled_z)
        # Concatenate samples horisontally
        fake_B = torch.cat([x for x in fake_B.data.cpu()], -1)
        img_sample = torch.cat((img_A, fake_B), -1)
        img_sample = img_sample.view(1, *img_sample.shape)
        # Concatenate with previous samples vertically
        img_samples = img_sample if img_samples is None else torch.cat((img_samples, img_sample), -2)
    save_image(img_samples, "images/%s/%s.png" % (opt.dataset_name, batches_done), nrow=8, normalize=True)
    generator.train()


    #再パラメータ化
def reparameterization(mu, logvar):
    std = torch.exp(logvar / 2)
    sampled_z = Variable(Tensor(np.random.normal(0, 1, (mu.size(0), opt.latent_dim))))
    z = sampled_z * std + mu
    return z


#############################################################

#  学習

##############################################################

# Adversarial loss
valid = 1
fake = 0

prev_time = time.time()
for epoch in range(opt.epoch, opt.n_epochs):
    for i, batch in enumerate(dataloader):
        # Set model input
        real_A = Variable(batch["A"].type(Tensor))
        # print(real_A.size())
        real_B = Variable(batch["B"].type(Tensor))

        # -------------------------------
        #  Train Generator and Encoder
        # -------------------------------

        optimizer_E.zero_grad()
        optimizer_G.zero_grad()

        # ----------
        # cVAE-GAN
        # ----------

        # Produce output using encoding of B (cVAE-GAN)
        # print(real_B.size())
        mu, logvar = encoder(real_B)     #######################ここでエラーが
        # print(mu)
        # print(logvar)
        encoded_z = reparameterization(mu, logvar)
        # print(encoded_z)
        fake_B = generator(real_A, encoded_z)

        # Pixelwise loss of translated image by VAE
        loss_pixel = mae_loss(fake_B, real_B)
        # Kullback-Leibler divergence of encoded B
        loss_kl = 0.5 * torch.sum(torch.exp(logvar) + mu ** 2 - logvar - 1)
        # Adversarial loss
        loss_VAE_GAN = D_VAE.compute_loss(fake_B, valid)

        # ---------
        # cLR-GAN
        # ---------

        # Produce output using sampled z (cLR-GAN)
        sampled_z = Variable(Tensor(np.random.normal(0, 1, (real_A.size(0), opt.latent_dim))))
        # print(sampled_z.size())
        _fake_B = generator(real_A, sampled_z)
        # print(_fake_B.size())
        # cLR Loss: Adversarial loss
        loss_LR_GAN = D_LR.compute_loss(_fake_B, valid)

        # ----------------------------------
        # Total Loss (Generator + Encoder)
        # ----------------------------------

        loss_GE = loss_VAE_GAN + loss_LR_GAN + opt.lambda_pixel * loss_pixel + opt.lambda_kl * loss_kl

        loss_GE.backward(retain_graph=True)
        optimizer_E.step()

        # ---------------------
        # Generator Only Loss
        # ---------------------

        # Latent L1 loss
        _mu, _ = encoder(_fake_B)
        loss_latent = opt.lambda_latent * mae_loss(_mu, sampled_z)

        loss_latent.backward()
        optimizer_G.step()

        # ----------------------------------
        #  Train Discriminator (cVAE-GAN)
        # ----------------------------------

        optimizer_D_VAE.zero_grad()

        loss_D_VAE = D_VAE.compute_loss(real_B, valid) + D_VAE.compute_loss(fake_B.detach(), fake)

        loss_D_VAE.backward()
        optimizer_D_VAE.step()

        # ---------------------------------
        #  Train Discriminator (cLR-GAN)
        # ---------------------------------

        optimizer_D_LR.zero_grad()

        loss_D_LR = D_LR.compute_loss(real_B, valid) + D_LR.compute_loss(_fake_B.detach(), fake)

        loss_D_LR.backward()
        optimizer_D_LR.step()

        # --------------
        #  Log Progress
        # --------------

        # Determine approximate time left
        batches_done = epoch * len(dataloader) + i
        batches_left = opt.n_epochs * len(dataloader) - batches_done
        time_left = datetime.timedelta(seconds=batches_left * (time.time() - prev_time))
        prev_time = time.time()

        # Print log
        sys.stdout.write(
            "\r[Epoch %d/%d] [Batch %d/%d] [D VAE_loss: %f, LR_loss: %f] [G loss: %f, pixel: %f, kl: %f, latent: %f] ETA: %s"
            % (
                epoch,
                opt.n_epochs,
                i,
                len(dataloader),
                loss_D_VAE.item(),
                loss_D_LR.item(),
                loss_GE.item(),
                loss_pixel.item(),
                loss_kl.item(),
                loss_latent.item(),
                time_left,
                
            )
        )
        D_VAE_loss.append(loss_D_VAE.item())
        LR_loss.append(loss_D_LR.item())
        G_loss.append(loss_GE.item())

        if batches_done % opt.sample_interval == 0:
            sample_images(batches_done)

        if epoch % opt.checkpoint_interval == 0:
        # Save model checkpoints
            torch.save(generator.state_dict(), "saved_models/%s/generator_%d.pth" % (opt.dataset_name, epoch))
            torch.save(encoder.state_dict(), "saved_models/%s/encoder_%d.pth" % (opt.dataset_name, epoch))
        # torch.save(D_VAE.state_dict(), "saved_models/%s/D_VAE_%d.pth" % (opt.dataset_name, epoch))
        # torch.save(D_LR.state_dict(), "saved_models/%s/D_LR_%d.pth" % (opt.dataset_name, epoch))

        
torch.save(generator.state_dict(), "saved_models/%s/generator_last.pth" % (opt.dataset_name))
torch.save(encoder.state_dict(), "saved_models/%s/encoder_last.pth" % (opt.dataset_name))

Namespace(epoch=0, n_epochs=90, dataset_name='sample', batch_size=16, lr=0.0002, b1=0.5, b2=0.999, n_cpu=3, img_height=256, img_width=256, channels=3, latent_dim=8, sample_interval=500, checkpoint_interval=40, lambda_pixel=10, lambda_latent=0.5, lambda_kl=0.01)
CUDAが使えます


C:\Users\toyou\anaconda3\lib\site-packages\torchvision\transforms\transforms.py:332: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


[Epoch 20/90] [Batch 1122/1125] [D VAE_loss: 1.390362, LR_loss: 1.154687] [G loss: 3.281095, pixel: 0.138301, kl: 0.001682, latent: 0.357105] ETA: 17:54:44.9996946.071144

In [ ]:
import pandas as pd
D_VAE_loss_DF=pd.DataFrame(D_VAE_loss,columns={"D_VAE_loss"})
LR_loss_DF=pd.DataFrame(LR_loss,columns={"LR_loss"})
G_loss_DF=pd.DataFrame(G_loss,columns={"G_loss"})


loss_all=pd.DataFrame()
loss_all=pd.concat([loss_all,D_VAE_loss_DF],axis=0)
loss_all=pd.concat([loss_all,LR_loss_DF],axis=1)
loss_all=pd.concat([loss_all,G_loss_DF],axis=1)


loss_all.to_csv("./bicycle_Loss.csv")